**Advanced Data Science capstone project** \
For this capstone project, we use a dataset regarding celestial object measurements and their classification into three possible candidates: a galaxy, a star or a quasar. Before we go further, we need to install pyspark, keras and tensorflow libraries for setting the model, as they are not already installed.

In [2]:
!pip install tensorflow==2.2
!pip install keras
!pip install pyspark==2.4.5

     |████████████████████████████████| 516.2 MB 6.8 kB/s  eta 0:00:01
     |████████████████████████████████| 26.1 MB 52.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 58.1 MB/s eta 0:00:01
     |████████████████████████████████| 454 kB 62.5 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 63.4 MB/s eta 0:00:01
  Attempting uninstall: gast
    Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.0
    Uninstalling scipy-1.5.0:
      Successfully uninstalled scipy-1.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorf

  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.1.0
    Uninstalling tensorflow-2.1.0:
      Successfully uninstalled tensorflow-2.1.0
     |████████████████████████████████| 217.8 MB 11 kB/s s eta 0:00:019
     |████████████████████████████████| 197 kB 55.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=978e170c84817ecfa03bf1afd80355caade93b66832a556ba125f87967b6ab12
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

Now we need to import few other necessary libraries that will be needed and add the dataset.

In [4]:
import numpy as np

#libraries for the machine learning model
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#libraries for the deep learning model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [5]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f1a0c12d7fb84344af027261a2aa9ba8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ARYhgLCYvhaUWh9CUw9--q2M6_-1sbXvh0g7wNZ5cew7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_f1a0c12d7fb84344af027261a2aa9ba8.get_object(Bucket='advancedatasciencecourse-donotdelete-pr-qfeni9xv77zjog',Key='Skyserver_12_30_2019 4_49_58 PM.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1237666301628060000,47.372545,0.820621,18.69254,17.13867,16.55555,16.34662,16.17639,4849,301,5,771,8168632633242440000,STAR,0.000115,7255,56597,832
1,1237673706652430000,116.303083,42.455980,18.47633,17.30546,17.24116,17.32780,17.37114,6573,301,6,220,9333948945297330000,STAR,-0.000093,8290,57364,868
2,1237671126974140000,172.756623,-8.785698,16.47714,15.31072,15.55971,15.72207,15.82471,5973,301,1,13,3221211255238850000,STAR,0.000165,2861,54583,42
3,1237665441518260000,201.224207,28.771290,18.63561,16.88346,16.09825,15.70987,15.43491,4649,301,3,121,2254061292459420000,GALAXY,0.058155,2002,53471,35
4,1237665441522840000,212.817222,26.625225,18.88325,17.87948,17.47037,17.17441,17.05235,4649,301,3,191,2390305906828010000,GALAXY,0.072210,2123,53793,74


We start doing some feature engineering as dropping the columns that are not relevant, as the ones involving "ID" or their coordinates, and vectorizing the brightness bands and indexing the labels.

In [6]:
df.rename(columns={'u': 'U-band','g': 'G-band','r': 'R-band','i': 'I-band','z': 'Z-band'}, inplace=True)
df.drop(['objid','specobjid','fiberid'],axis=1,inplace=True)

We convert the pandas Dataframe into a PySpark dataframe and establish the train and test datasets. 

In [24]:
df_ml=spark.createDataFrame(df)
df_ml.createOrReplaceTempView("df_ml")
spark.sql("SELECT class,count(*) from df_ml group by class").show()

+------+--------+
| class|count(1)|
+------+--------+
|GALAXY|   51323|
|   QSO|   10581|
|  STAR|   38096|
+------+--------+



In [26]:
(train_set,test_set) = df_ml.randomSplit([0.7, 0.3], 1234)

We set the stages for the pipeline.

In [35]:
feature_list=df_ml.columns
feature_list.remove('class')
featureAssembler=VectorAssembler(inputCols=feature_list,outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="norm_features", p=1.0)
classIndexer=StringIndexer(inputCol="class", outputCol="class_index")

Considering our problem a classification model seems appropiate, for example GBT one.

In [36]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier

Dtree=DecisionTreeClassifier(labelCol='class_index', featuresCol='norm_features')
Randforest=RandomForestClassifier(labelCol='class_index', featuresCol='norm_features',numTrees=5)

We set the pipeline and train the model.

In [37]:
pipe=Pipeline(stages=[featureAssembler,normalizer,classIndexer,Dtree])
pipe1=Pipeline(stages=[featureAssembler,normalizer,classIndexer,Randforest])

In [38]:
ml_model = pipe.fit(train_set)
ml_model1 = pipe1.fit(train_set)

We now evalute the model and it's precision.

In [39]:
pred_ml = ml_model.transform(test_set)
pred_ml1 = ml_model1.transform(test_set)

In [40]:
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class_index")
    
ml_accuracy=binEval.evaluate(pred_ml)
ml_accuracy1=binEval.evaluate(pred_ml1)
print('Accuracy for Decision Tree: ',ml_accuracy)
print('Accuracy for Random Forest: ',ml_accuracy1)

Accuracy for Decision Tree:  0.9780354888017545
Accuracy for Random Forest:  0.9626503621984449


We manage to get a 97% of accuracy determining the object via the machine learning model. Now we build the deep learning model for comparison, also, we need to reset the train and test set.

In order to set the train and test set for the neural network we need to convert again the pyspark dataframe to a pandas dataframe.

In [443]:
df.shape

(100000, 15)

In [444]:
df_labels=df[['class']]
df_features=df.drop('class',axis=1)

In [445]:
from sklearn.preprocessing import LabelEncoder

labs=LabelEncoder().fit_transform(df_labels)

df_labels['class_encoded']=labs
df_labels.drop('class',axis=1,inplace=True)
df_labels.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,class_encoded
0,2
1,2
2,2
3,0
4,0


In [498]:
samples=df_features.shape[0]
dimensions=df_features.shape[1]

feature_train, feature_test, label_train, label_test = train_test_split(df_features, df_labels, test_size=0.3)
feature_train, feature_validation, label_train, label_validation = train_test_split(feature_train, label_train, test_size=0.3)

We build the neural network and train it.

In [501]:
epochs = 5

nn_model=Sequential()
nn_model.add(Dense(15, input_shape=(dimensions, ), activation='softmax'))
nn_model.add(Dropout(0.3))
#nn_model.add(Dense(15, activation='softmax'))
nn_model.add(Dense(3, activation='softmax'))
nn_model.compile(loss='binary_crossentropy', optimizer="adam")

In [502]:
nn_model.fit(feature_train, label_train, epochs=epochs, batch_size=60, validation_data=(feature_validation, label_validation), verbose=0, shuffle=True)

It's time to test the accuracy of the neural network.

In [503]:
test_pred=nn_model.predict(feature_test)

correct=0
for i in range(len(test_pred)):
    label_pred=np.argmax(test_pred[i])
    if label_pred==label_test['class_encoded'].tolist()[i]:
        correct+=1
        
print('Accuracy of the neural network is: ',correct/len(test_pred))

Accuracy of the neural network is:  0.5137
